This notebook investigates the financial impact of https://skoove.atlassian.net/wiki/spaces/DEV/pages/3755933700/Incident+postmortem+Paypal+2024-04-17

In [108]:
import pandas as pd
# Don't truncate output
pd.set_option('display.max_columns', None)


In [101]:
# Exported from https://dashboard.stripe.com/payments
df_src = pd.read_csv('data/stripe_payments_2024-04-08_2024-05-06.csv', quotechar='"', skipinitialspace=True)
# Column names have leading/trailing spaces -> strip them
df_src.columns = df_src.columns.str.strip()

In [95]:
df_src.head()

,id,Created date (UTC),Amount,Amount Refunded,Currency,Captured,Converted Amount,Converted Amount Refunded,Converted Currency,Description,Fee,Is Link,Mode,PaymentIntent ID,Payment Source Type,Refunded date (UTC),Statement Descriptor,Status,Seller Message,Taxes On Fee,Interchange Costs,Merchant Service Charge,Card ID,Card Name,Card Address Line1,Card Address Line2,Card Address City,Card Address State,Card Address Country,Card Address Zip,Card AVS Line1 Status,Card AVS Zip Status,Card Brand,Card CVC Status,Card Exp Month,Card Exp Year,Card Fingerprint,Card Funding,Card Issue Country,Card Last4,Card Tokenization Method,Customer ID,Customer Description,Customer Email,Customer Phone,Shipping Name,Shipping Address Line1,Shipping Address Line2,Shipping Address City,Shipping Address State,Shipping Address Country,Shipping Address Postal Code,Disputed Amount,Dispute Date (UTC),Dispute Evidence Due (UTC),Dispute Reason,Dispute Status,Invoice ID,Invoice Number,Checkout Session ID,Checkout Custom Field 1 Key,Checkout Custom Field 1 Value,Checkout Custom Field 2 Key,Checkout Custom Field 2 Value,Checkout Custom Field 3 Key,Checkout Custom Field 3 Value,Checkout Line Item Summary,Checkout Promotional Consent,Checkout Terms of Service Consent,Client Reference ID,Payment Link ID,UTM Campaign,UTM Content,UTM Medium,UTM Source,UTM Term,Application Fee,Application ID,Destination,Transfer,Transfer Group,user_id (metadata),source (metadata),Purchaser Email (metadata),Gift Up Order Id (metadata),Recipient Name (metadata),Gift Card Code (metadata),Gift Card Link (metadata),Recipient Email (metadata),Purchaser Name (metadata)
0,ch_3PDVunGXe0XeY11p1UWZqo8n,2024-05-06 18:05:29,"59,99","0,00",usd,False,"59,99","0,00",usd,Invoice ED065010-0003,"0,00",False,Live,pi_3PDVunGXe0XeY11p1eTJLnLo,card,NaN,SKOOVE,Failed,The bank returned the decline code `insufficie...,"0,00",NaN,NaN,card_1O9WDgGXe0XeY11pE5kKHzwi,John M Nielsen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MasterCard,NaN,11.0,2025.0,I76fyZwUcM9ChY1q,credit,US,274.0,NaN,cus_OxR5yAA1BB0NnE,NaN,john4omega@yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,in_1PDUy9GXe0XeY11pTODEvZ0R,ED065010-0003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ch_3PBBe6GXe0XeY11p2uiwOdqJ,2024-05-06 18:02:49,"39,99","0,00",eur,False,"39,99","0,00",eur,Invoice C23EF8F3-0011,"0,00",False,Live,pi_3PBBe6GXe0XeY11p2gr4LpDS,card,NaN,SKOOVE,Failed,The bank returned the decline code `insufficie...,"0,00",NaN,NaN,card_1JqBZbGXe0XeY11p5rUXPZRc,Samu Hokkanen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visa,NaN,10.0,2025.0,xlpq40UwUD5MJxqp,credit,FI,9213.0,NaN,cus_KVBxW1UP6FVSon,NaN,samu.hokkanen@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,in_1PBAhdGXe0XeY11pT3uUMZJR,C23EF8F3-0011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,py_3PDVkfGXe0XeY11p1ng1Z3gJ,2024-05-06 17:55:05,"59,99","0,00",eur,True,"59,99","0,00",eur,Invoice 03CB504D-0002,"2,40",False,Live,pi_3PDVkfGXe0XeY11p1qkDDya2,paypal,NaN,NaN,Paid,Payment complete.,"0,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cus_PV9Epbowcsj35v,NaN,massimofinotto@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,in_1PDUnKGXe0XeY11pTuahnxDS,03CB504D-0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ch_3PBEKbGXe0XeY11p2ZtBjQs1,2024-05-06 17:54:49,"39,99","0,00",eur,False,"39,99","0,00",eur,Invoice 76972B49-0016,"0,00",False,Live,pi_3PBEKbGXe0XeY11p2rHMtmx2,card,NaN,SKOOVE,Failed,The bank returned the decline code `try_again_...,"0,00",NaN,NaN,card_1HAuvlGXe0XeY11pwK2zE0XW,Gian Pietro Severi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MasterCard,NaN,6.0,2021.0,Tr34yJ7QdN8HO9wU,prepaid,IT,8876.0,NaN,cus_HkPlNBA8P716mM,NaN,katyandra49@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,in_1PBDOLGXe0XeY11

In [96]:
df = df_src[['id', 'Created date (UTC)', 'Amount', 'Amount Refunded', 'Currency', 'Description', 'Status', 'Customer Email', 'Payment Source Type', 'Seller Message']]

# Data cleaning (loc is used to change in place, without triggering SettingWithCopyWarning)
df.loc[:, 'Created date (UTC)'] = pd.to_datetime(df['Created date (UTC)'])
df.loc[:, 'Amount'] = pd.to_numeric(df['Amount'].str.replace(',', '.'), errors='coerce')

In [97]:
df_failed_paypal_during_incident = df[
	(df['Created date (UTC)'] >= pd.to_datetime('2024-04-08'))
	& (df['Created date (UTC)'] <= pd.to_datetime('2024-04-24'))
	& (df['Status'] == 'Failed')
	& (df['Payment Source Type'] == 'paypal')
]

In [98]:
df_succeeded_after_incident = df[
	(df['Created date (UTC)'] >= pd.to_datetime('2024-04-24'))
	& (df['Status'] == 'Paid')
]

2 Customers who were affected by the PayPal outage have a successful payment after the incident was over:

In [99]:
df_healed = df_succeeded_after_incident[
	df_succeeded_after_incident['Customer Email'].isin(df_failed_paypal_during_incident['Customer Email'])]
df_healed

,id,Created date (UTC),Amount,Amount Refunded,Currency,Description,Status,Customer Email,Payment Source Type,Seller Message
1967,py_3P7f7mGXe0XeY11p0PlCtpN2,2024-04-24 02:42:49,59.99,"0,00",usd,Invoice 2A0A9B35-0002,Paid,infobetancoservices@gmail.com,paypal,Payment complete.
1973,py_3P6NezGXe0XeY11p1LqfZHdH,2024-04-24 01:51:52,59.99,"0,00",usd,Invoice 861DA2F8-0002,Paid,ben.peter.leonard@gmail.com,paypal,Payment complete.


432 failed payments:

In [123]:
df_permanently_failed = df_failed_paypal_during_incident[
	~df_failed_paypal_during_incident['Customer Email'].isin(df_healed['Customer Email'])]
df_permanently_failed.shape


(432, 10)

Payments were repeated up to 5 times:

In [110]:
df_permanently_failed.groupby('Customer Email').size().sort_values(ascending=False).head()

Customer Email
12504mee@gmail.com             5
info@tretbar.net               5
markusanthonyjr26@gmail.com    5
madisoncranford4@gmail.com     5
luna1stars2@yahoo.com          5
dtype: int64

In [119]:
df_permanently_failed_unique_emails = df_permanently_failed.drop_duplicates(subset=('Customer Email', 'Currency', 'Amount'))

# Total volume of failed payments, by currency

In [135]:
df_permanently_failed_unique_emails.groupby('Currency')['Amount'].sum().to_frame()

,Amount
Currency,
eur,2489.63
gbp,249.95
usd,4529.31


# Failed purchase attempts, by offer (currency and amount):

In [127]:
df_permanently_failed_unique_emails.groupby(['Currency', 'Amount'])['Amount'].count().to_frame()

Amount
Currency Amount        
eur      29.99        1
         59.99       32
         89.99        1
         149.99       3
gbp      49.99        5
usd      29.99        2
         59.99       62
         149.99       5

# Questions
- What kinds of subscriptions do these failed transactions refer to? Which of these were failed renewals?
- Did any of the failed purchase potential customers get in touch? Did they pay some other way, outside of Stripe?

All emails of users with failed PayPal payments:

In [134]:
for email in df_permanently_failed_unique_emails['Customer Email']:
	print(email)

dagjmg01930@gmail.com
rayvon.garvin@hotmail.com
theo0329@yahoo.com
markusanthonyjr26@gmail.com
jarodlutzz@gmail.com
juliengrabowski@googlemail.com
superakshay@gmail.com
nanettecook@gmail.com
stefaniepietsch@hotmail.com
h9jhjx6bxh@privaterelay.appleid.com
amanda.anderson0251@gmail.com
doris.niedernolte@gmail.com
mail@gala-maikath.de
cpznvrnqsk@privaterelay.appleid.com
ciao.rob@gmail.com
menezes02@gmail.com
ctcellan@googlemail.com
olminator@hotmail.com
carlosjtorres@hotmail.com
kaylobwalker1@gmail.com
meghan_horwood@hotmail.com
6hf5v7vpmc@privaterelay.appleid.com
analagu@yahoo.es
tschessi@gmx.net
treehug@gmail.com
lovinmt4916@gmail.com
madisoncranford4@gmail.com
scalisi.durance@gmail.com
lopoukhova@gmail.com
ju.junge@yahoo.com
saralubura63@gmail.com
regina_llagas@me.com
agnesk90@gmail.com
elenore2016@gmail.com
joerg.wingens@web.de
tenhundfeldkody@gmail.com
jasmine.m.duren@gmail.com
nuria_0711@hotmail.com
qczs647byw@privaterelay.appleid.com
mslegaspi84@gmail.com
leonardodiazcutie@gmail.co